# Import Data

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

random_state = 3

In [2]:
Boosting_data = pd.read_csv("Data/BoostingData.csv")
Boosting_data

,Index,DT,Energy,Month,DayOfWeek,BankHols,RetailHols,Trend2010,AvgDB,LagDB,...,AvgWind,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,0,1/1/17,242.69,1.0,7.0,0.0,0.0,7.00,46.77,36.66,...,5.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1/2/17,257.06,1.0,1.0,0.0,0.0,7.01,45.10,46.77,...,5.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1/3/17,265.35,1.0,2.0,0.0,0.0,7.01,45.37,45.10,...,5.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1/4/17,253.79,1.0,3.0,0.0,0.0,7.01,48.87,45.37,...,8.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1/5/17,305.91,1.0,4.0,0.0,0.0,7.01,34.49,48.87,...,3.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1090,12/27/19,262.49,12.0,5.0,0.0,0.0,9.99,47.83,43.47,...,2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1091,1091,12/28/19,238.96,12.0,6.0,0.0,0.0,9.99,51.19,47.83,...,1.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1092,1092,12/29/19,240.27,12.0,7.0,0.0,0.0,9.99,53.24,51.19,...,3.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1093,1093,12/30/19,235.10,12.0,1.0,0.0,0.0,10.00,62.12,53.24,...,9.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
X = Boosting_data.values[:,3:]
y = Boosting_data.values[:,2]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)

# Model